<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_Agnostic_Run.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# Run Agnostic App  📒
***Create your own Tapis App***

by Silvia Mazzoni, DesignSafe, 2025

This path will run the executable you want, with some bells and whitles.

This is a step-by-step guide to help you write your own **Tapis v3 App** — from defining the app, to registering it, and running it. 

This is a practical walkthrough for defining and deploying an **HPC** app using the Tapis v3 API.


## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
import json

In [2]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


---
## Connect to Tapis

In [3]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-21T09:38:01+00:00
 Token expires in: 3:55:19.530355
-- LOG IN SUCCESSFUL! --


---
### Get username

In [4]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_username.py'])

In [5]:
username = OpsUtils.get_tapis_username(t)
print('username',username)

username silvia


---
## Configure App

In [6]:
app_id = 'agnostic-app-test'

#### List the app schema

In [7]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
print(appMetaData)


containerImage: tapis://designsafe.storage.default/silvia/apps/agnostic-app-test/0.0.29/agnostic-app-test.zip
created: 2025-08-21T05:42:14.691906Z
deleted: False
description: Run any Executable
enabled: True
id: agnostic-app-test
isPublic: False
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: stampede3
cmdPrefix: None
coresPerNode: 48
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: ${JobWorkingDir}
execSystemId: stampede3
execSystemInputDir: ${JobWorkingDir}
execSystemLogicalQueue: skx
execSystemOutputDir: ${JobWorkingDir}
fileInputArrays: []
fileInputs: [
autoMountLocal: True
description: Input directory that includes the input script as well as any other required files. Example input is in tapis://designsafe.storage.community/app_examples/opensees/OpenSeesMP
envKey: inputDi

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

---
### Initialize

In [8]:
tapisInputAll = {}

---
### SLURM-Specific Input

In [9]:
tapisInputAll["maxMinutes"] = 6

tapisInputAll["execSystemId"] = "stampede3"
tapisInputAll["execSystemLogicalQueue"] = "skx-dev"
tapisInputAll["nodeCount"] = 1
tapisInputAll["coresPerNode"] = 48
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='Work' # Options: MyData or Work

---
### App-Specific Input

In [10]:
print('app_id:',app_id)

app_id: agnostic-app-test


### OpenSees

In [11]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'OpenSees'
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-express',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- OpenSees_opensees-express --

Creating job_description
job_description {'name': 'OpenSees_opensees-express', 'maxMinutes': 6, 'appId': 'opensees-express', 'appVersion': 'latest', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'envVariables': [{'key': 'mainProgram', 'value': 'OpenSees'}, {'key': 'tclScript', 'value': 'Ex1a.Canti2D.Push.tcl'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}'}
Submitting Job
Job submitted! ID: d8eddc68-2b70-4669-93e7-25067b95c103-007
job_start_time: 1755754966.108333

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.04 sec	 Current Status: PENDING
	 Elapsed job time: 2.07 sec	 Current Status: PROCESSING_INPUTS		(PENDING took 1.03 sec)
	 Elapsed job time: 4.18 sec	 Current Status: STAGING_I

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (d8eddc68-2b70-4669-93e7-25067b95c103-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (d8eddc68-2b70-4669-93e7-25067b95c10…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (d8eddc68-2b70-4669-93e7-25067b9…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (d8eddc68-2b70-4669-93e7-25067b95c10…

...


 -- OpenSees_agnostic-app-test_zipFolderOut --

Creating job_description
job_description {'name': 'OpenSees_agnostic-app-test_zipFolderOut', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'OpenSees'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.tcl'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'True'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDat

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (34062500-8f84-4a3f-83ce-86b98a6151a8-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (34062500-8f84-4a3f-83ce-86b98a6151a…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (34062500-8f84-4a3f-83ce-86b98a6…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (34062500-8f84-4a3f-83ce-86b98a6151a…

### OpenSeesMP

In [12]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'OpenSeesMP'
tapisInput['Main Script'] = 'Ex1a_many.Canti2D.Push.mp.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-mp-s3',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- OpenSeesMP_opensees-mp-s3 --

Creating job_description
job_description {'name': 'OpenSeesMP_opensees-mp-s3', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'opensees-mp-s3', 'appVersion': 'latest', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Script', 'arg': 'Ex1a_many.Canti2D.Push.mp.tcl'}], 'envVariables': [], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}'}
Submitting Job
Job submitted! ID: a49e1546-5eb3-4f4f-a49b-fa14af91f244-007
job_start_time: 1755755252.1832323

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.05 sec	 Current Status: 

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (a49e1546-5eb3-4f4f-a49b-fa14af91f244-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (a49e1546-5eb3-4f4f-a49b-fa14af91f24…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (a49e1546-5eb3-4f4f-a49b-fa14af9…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (a49e1546-5eb3-4f4f-a49b-fa14af91f24…

...


 -- OpenSeesMP_agnostic-app-test_zipFolderOut --

Creating job_description
job_description {'name': 'OpenSeesMP_agnostic-app-test_zipFolderOut', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'OpenSeesMP'}, {'name': 'Main Script', 'arg': 'Ex1a_many.Canti2D.Push.mp.tcl'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'True'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (f211e8a1-7434-4df4-ab80-7a4be4aa8d5c-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (f211e8a1-7434-4df4-ab80-7a4be4aa8d5…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (f211e8a1-7434-4df4-ab80-7a4be4a…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (f211e8a1-7434-4df4-ab80-7a4be4aa8d5…

### OpenSeesSP

In [14]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'OpenSeesSP'
tapisInput['Main Script'] = 'simpleSP.tcl'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in ['opensees-sp-s3',app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- OpenSeesSP_opensees-sp-s3 --

Creating job_description
job_description {'name': 'OpenSeesSP_opensees-sp-s3', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'opensees-sp-s3', 'appVersion': 'latest', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Script', 'arg': 'simpleSP.tcl'}], 'envVariables': [], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}'}
Submitting Job
Job submitted! ID: 7dbdef22-5044-41fa-9944-d9a406946fba-007
job_start_time: 1755757636.8082068

Real-Time Job-Status Updates...
--------------------
	 Elapsed job time: 1.05 sec	 Current Status: PENDING
	 Elapsed

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (7dbdef22-5044-41fa-9944-d9a406946fba-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (7dbdef22-5044-41fa-9944-d9a406946fb…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (7dbdef22-5044-41fa-9944-d9a4069…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (7dbdef22-5044-41fa-9944-d9a406946fb…

...


 -- OpenSeesSP_agnostic-app-test_zipFolderOut --

Creating job_description
job_description {'name': 'OpenSeesSP_agnostic-app-test_zipFolderOut', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'OpenSeesSP'}, {'name': 'Main Script', 'arg': 'simpleSP.tcl'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'True'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (01a39616-1311-4ef4-abde-6eae13752174-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (01a39616-1311-4ef4-abde-6eae1375217…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (01a39616-1311-4ef4-abde-6eae137…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (01a39616-1311-4ef4-abde-6eae1375217…

### OpenSeesPy

In [15]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'python'
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- python_agnostic-app-test --

Creating job_description
job_description {'name': 'python_agnostic-app-test', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'python'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.py'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'False'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}'}
Submi

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (60464055-8b4b-4aa3-a1e5-2cc4c6dd9016-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (60464055-8b4b-4aa3-a1e5-2cc4c6dd901…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (60464055-8b4b-4aa3-a1e5-2cc4c6d…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (60464055-8b4b-4aa3-a1e5-2cc4c6dd901…


 -- python_agnostic-app-test_zipFolderOut --

Creating job_description
job_description {'name': 'python_agnostic-app-test_zipFolderOut', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'python'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.py'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'True'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${Jo

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (aa336728-6121-4f89-b114-e1ed60cd5f7b-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (aa336728-6121-4f89-b114-e1ed60cd5f7…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (aa336728-6121-4f89-b114-e1ed60c…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (aa336728-6121-4f89-b114-e1ed60cd5f7…

### OpenSeesPy - mpi

In [16]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'python'
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.mpi.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- python_agnostic-app-test --

Creating job_description
job_description {'name': 'python_agnostic-app-test', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'python'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.mpi.py'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'False'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}'}
S

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (a770978a-c3fd-4759-b1c0-afe2c37d13a0-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (a770978a-c3fd-4759-b1c0-afe2c37d13a…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (a770978a-c3fd-4759-b1c0-afe2c37…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (a770978a-c3fd-4759-b1c0-afe2c37d13a…


 -- python_agnostic-app-test_zipFolderOut --

Creating job_description
job_description {'name': 'python_agnostic-app-test_zipFolderOut', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'python'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.mpi.py'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'True'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (40764c7f-62f5-4c82-9c2a-304408a2baf7-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (40764c7f-62f5-4c82-9c2a-304408a2baf…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (40764c7f-62f5-4c82-9c2a-304408a…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (40764c7f-62f5-4c82-9c2a-304408a2baf…

### OpenSeesPy - mpi4py

In [17]:
tapisInput = tapisInputAll.copy()

tapisInput['storage_system'] = 'CommunityData'
tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 

tapisInput['Main Program'] = 'python'
tapisInput['Main Script'] = 'Ex1a.Canti2D.Push.mpi4py.py'

tapisInput['CommandLine Arguments'] = '[33,22]'

# -----------------------------------------------------
for hereApp in [app_id]:
    here_tapisInput = tapisInput.copy()
    here_tapisInput["appId"] = hereApp 
    here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"]
    print(f'\n -- {here_tapisInput['name']} --\n')
    jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
    if hereApp == app_id:
        here_tapisInput['zipFolderOut'] = 'True'
        here_tapisInput["name"] = here_tapisInput['Main Program'] + '_' + here_tapisInput["appId"] + '_' + 'zipFolderOut'
        print(f'\n -- {here_tapisInput['name']} --\n')
        jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# -----------------------------------------------------


 -- python_agnostic-app-test --

Creating job_description
job_description {'name': 'python_agnostic-app-test', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'python'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.mpi4py.py'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'False'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}'

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (e2baaec6-2c59-43e6-ab68-ae5a9715d645-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (e2baaec6-2c59-43e6-ab68-ae5a9715d64…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (e2baaec6-2c59-43e6-ab68-ae5a971…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (e2baaec6-2c59-43e6-ab68-ae5a9715d64…


 -- python_agnostic-app-test_zipFolderOut --

Creating job_description
job_description {'name': 'python_agnostic-app-test_zipFolderOut', 'execSystemId': 'stampede3', 'execSystemLogicalQueue': 'skx-dev', 'maxMinutes': 6, 'nodeCount': 1, 'coresPerNode': 48, 'appId': 'agnostic-app-test', 'appVersion': '0.0.29', 'fileInputs': [{'name': 'Input Directory', 'sourceUrl': 'tapis://designsafe.storage.community/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples'}], 'parameterSet': {'appArgs': [{'name': 'Main Program', 'arg': 'python'}, {'name': 'Main Script', 'arg': 'Ex1a.Canti2D.Push.mpi4py.py'}, {'name': 'CommandLine Arguments', 'arg': '[33,22]'}], 'envVariables': [{'key': 'zipFileIn', 'value': ''}, {'key': 'zipFolderOut', 'value': 'True'}, {'key': 'requirementFile', 'value': ''}], 'schedulerOptions': [{'name': 'TACC Allocation', 'arg': '-A DS-HPC1'}]}, 'archiveSystemId': 'stampede3', 'archiveSystemDir': 'HOST_EVAL($WORK)/tapis-jobs-archive/${JobCreateDat

Accordion(children=(Output(),), selected_index=0, titles=('Job STATUS   (d46e382c-aaeb-4f9d-ac34-415e85b09875-…

Accordion(children=(Output(),), selected_index=0, titles=('Job Metadata   (d46e382c-aaeb-4f9d-ac34-415e85b0987…

Accordion(children=(Output(),), selected_index=0, titles=('Job History Data   (d46e382c-aaeb-4f9d-ac34-415e85b…

Accordion(children=(Output(),), selected_index=0, titles=('Job Filedata   (d46e382c-aaeb-4f9d-ac34-415e85b0987…